In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from malnis import show
from nltk.tokenize import word_tokenize
from tqdm.auto import tqdm
import scipy.sparse as sp
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA, SparsePCA
from sklearn.metrics import accuracy_score, f1_score, auc, roc_auc_score, average_precision_score, PrecisionRecallDisplay
import numpy as np
from sentence_transformers import SentenceTransformer

/home/jarobyte/envs/malnis/lib/python3.8/site-packages/huggingface_hub/snapshot_download.py:6: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  warnings.warn(


In [2]:
pwd

'/home/jarobyte/malnis_dataset/notebooks'

In [3]:
data = pd.read_pickle("../data/labels.pkl")#.head(50)
show(data)

(391, 8)


,query,document,summary,r1,r2,rl,sentences,relevance
36,"For many types of machine learning algorithms,...","field in computer science, and health informat...",[While the techniques for neural networks are ...,0.594595,0.485714,0.594595,"[field in computer science, and health informa...","[False, False, False, False, False, False, Fal..."
38,We consider the problem of learning good traje...,"field in computer science, and health informat...",[[71] considered the problem of learning good ...,0.646766,0.478571,0.636816,"[field in computer science, and health informa...","[False, False, False, False, False, False, Fal..."
136,We study in this work the importance of depth ...,Deep Learning Based Text Classification: A Com...,[[57] show that deep models indeed outperform ...,0.533333,0.403101,0.533333,[Deep Learning Based Text Classification: A Co...,"[False, False, False, False, False, False, Fal..."
5,We investigate a lattice-structured LSTM model...,1041-4347 (c) 2020 IEEE. Personal use is permi...,[Zhang and Yang [149] proposed a lattice-struc...,0.478261,0.393162,0.478261,"[1041-4347 (c) 2020 IEEE., Personal use is per...","[False, False, False, False, False, False, Fal..."
316,"HDBSCAN*, a state-of-the-art density-based hie...",PVLDB Reference Format: Antonio Cavalcante Ara...,"[In [6], the authors proposed RNG-HDBSCAN*, a ...",0.539062,0.350365,0.507812,[PVLDB Reference Format: Antonio Cavalcante Ar...,"[False, False, False, False, False, False, Fal..."


In [4]:
model = SentenceTransformer(
    "sbert"
#     'all-MiniLM-L6-v2', 
#     cache_folder = "../assets"
#     "../cache/huggingface/transformers/"
#     cache_folder = "../cache/huggingface/transformers"
)
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [5]:
query_embeddings = model.encode(data["query"])
query_embeddings.shape

(391, 384)

In [ ]:
dims = query_embeddings.shape[1]
sentence_embeddings = [
    model.encode(l)#.toarray() 
    for l in tqdm(data.sentences)
]
all([l.shape[1] == dims for l in sentence_embeddings])

  0%|          | 0/391 [00:00<?, ?it/s]

In [ ]:
query_embeddings.shape[0] == len(sentence_embeddings)

In [ ]:
sum([len(l) for l in tqdm(data.sentences)])

In [ ]:
np.tile(query_embeddings[0], (4, 1)).shape

In [ ]:
train = [
#     ((sp.csr_matrix(np.ones([l.shape[0],1])) * q) - l).power(2)
#     (q - l)**2
    np.concatenate([np.tile(q, (l.shape[0], 1)), l], axis = 1)
    for q, l in zip(tqdm(query_embeddings), sentence_embeddings)
]
len(train)

In [ ]:
X = np.concatenate(train)#.toarray().T#.squeeze()
X.shape

In [ ]:
Y = np.array([y for l in tqdm(data.relevance) for y in l])
Y.shape

In [ ]:
Y.mean()

In [ ]:
%%time
lr = LogisticRegression(
    C = 10**-2
)
lr.fit(X, Y)

predictions = lr.predict_proba(X)

# predictions.sum()
predictions.shape

In [ ]:
# %%time
# lr = LogisticRegression(
# #     C = 10**-2
# )
# lr.fit(X, Y)

# predictions = lr.predict_proba(X)

# # predictions.sum()
# predictions.shape

In [ ]:
# %%time
# lr = SVC(
# #     C = 10**-2
# )
# lr.fit(X, Y)

# predictions = lr.predict_proba(X)

# # predictions.sum()
# predictions.shape

In [ ]:
plt.hist(predictions[:, 1])
# plt.xlim(-0.1, 1.1)
plt.show()

In [ ]:
roc_auc_score(Y, predictions[:, 1])

In [ ]:
average_precision_score(Y, predictions[:, 1])

In [ ]:
display = PrecisionRecallDisplay.from_estimator(
    lr, X, Y, name="Logistic Regression"
)